In [ ]:
import numpy as np
import pandas as pd
import scipy.optimize

file_path = input("Enter the file path containing transportation problem data: ")
data = pd.read_csv(file_path, sep=";", header=None)

# Load supply, demand, and cost matrix from input file
cost_matrix = data.iloc[:-1, :-1].values
supply = data.iloc[:-1, -1].values
demand = data.iloc[-1, :-1].values

print("Cost Matrix:\n", cost_matrix)
print("Supply:", supply)
print("Demand:", demand)

# Methods to find the initial feasible solution and transportation simplex algorithm

def northwest_corner_rule(supply, demand):
    """
    Applies the Northwest Corner Rule to compute an initial feasible allocation.

    Args:
        supply (array): Array of supply values.
        demand (array): Array of demand values.

    Returns:
        array: Allocation matrix.
    """
    m, n = len(supply), len(demand)
    allocation = np.zeros((m, n))
    i, j = 0, 0
    while i < m and j < n:
        allocation[i, j] = min(supply[i], demand[j])
        if supply[i] < demand[j]:
            demand[j] -= supply[i]
            i += 1
        else:
            supply[i] -= demand[j]
            j += 1
    return allocation

def minimum_cost_method(cost_matrix, supply, demand):
    """
    Finds an initial allocation by iteratively selecting cells with the lowest cost.

    Args:
        cost_matrix (array): Cost matrix.
        supply (array): Array of supply values.
        demand (array): Array of demand values.

    Returns:
        array: Allocation matrix.
    """
    rows, cols = cost_matrix.shape
    allocation = np.zeros((rows, cols), dtype=float)
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    cost_matrix_copy = cost_matrix.astype(float)

    while np.any(supply_copy > 0) and np.any(demand_copy > 0):
        min_cost_index = np.unravel_index(np.argmin(cost_matrix_copy, axis=None), cost_matrix_copy.shape)
        row, col = min_cost_index

        allocation_value = min(supply_copy[row], demand_copy[col])
        allocation[row, col] = allocation_value

        supply_copy[row] -= allocation_value
        demand_copy[col] -= allocation_value
        cost_matrix_copy[row, col] = np.inf

    return allocation

def minimum_row_cost_method(cost_matrix, supply, demand):
    """
    Computes an initial allocation by considering minimum-cost cells row by row.

    Args:
        cost_matrix (array): Cost matrix.
        supply (array): Array of supply values.
        demand (array): Array of demand values.

    Returns:
        array: Allocation matrix.
    """
    rows, cols = cost_matrix.shape
    allocation = np.zeros((rows, cols), dtype=float)
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    cost_matrix_copy = cost_matrix.astype(float)

    for row in range(rows):
        while supply_copy[row] > 0 and np.any(demand_copy > 0):
            col = np.argmin(cost_matrix_copy[row, :])
            allocation_value = min(supply_copy[row], demand_copy[col])
            allocation[row, col] = allocation_value

            supply_copy[row] -= allocation_value
            demand_copy[col] -= allocation_value
            cost_matrix_copy[row, col] = np.inf

    return allocation

def vogel_method(cost_matrix, supply, demand):
    """
    Implements Vogel's Approximation Method to compute an initial feasible allocation.

    Args:
        cost_matrix (array): Cost matrix.
        supply (array): Array of supply values.
        demand (array): Array of demand values.

    Returns:
        array: Allocation matrix.
    """
    rows, cols = cost_matrix.shape
    allocation = np.zeros((rows, cols), dtype=float)
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    cost_matrix_copy = cost_matrix.astype(float)

    while np.any(supply_copy > 0) and np.any(demand_copy > 0):
        row_penalties = np.full(rows, np.inf)
        col_penalties = np.full(cols, np.inf)

        for i in range(rows):
            if supply_copy[i] > 0:
                valid_costs = cost_matrix_copy[i, demand_copy > 0]
                if len(valid_costs) > 1:
                    row_penalties[i] = np.partition(valid_costs, 1)[1] - np.min(valid_costs)

        for j in range(cols):
            if demand_copy[j] > 0:
                valid_costs = cost_matrix_copy[supply_copy > 0, j]
                if len(valid_costs) > 1:
                    col_penalties[j] = np.partition(valid_costs, 1)[1] - np.min(valid_costs)

        if np.min(row_penalties) < np.min(col_penalties):
            row = np.argmin(row_penalties)
            col = np.argmin(cost_matrix_copy[row, :])
        else:
            col = np.argmin(col_penalties)
            row = np.argmin(cost_matrix_copy[:, col])

        allocation_value = min(supply_copy[row], demand_copy[col])
        allocation[row, col] = allocation_value

        supply_copy[row] -= allocation_value
        demand_copy[col] -= allocation_value
        cost_matrix_copy[row, col] = np.inf

    return allocation

def transportation_simplex(cost_matrix, supply, demand):
    """
    Solves the transportation problem using the simplex method.

    Args:
        cost_matrix (array): Cost matrix.
        supply (array): Array of supply values.
        demand (array): Array of demand values.

    Returns:
        tuple: Optimal allocation matrix and total cost.
    """
    rows, cols = cost_matrix.shape
    c = cost_matrix.flatten()

    supply_constraints = np.zeros((rows, rows * cols))
    for i in range(rows):
        supply_constraints[i, i * cols:(i + 1) * cols] = 1

    demand_constraints = np.zeros((cols, rows * cols))
    for j in range(cols):
        demand_constraints[j, j::cols] = 1

    A_eq = np.vstack([supply_constraints, demand_constraints])
    b_eq = np.concatenate([supply, demand])

    result = scipy.optimize.linprog(c, A_eq=A_eq, b_eq=b_eq, bounds=(0, None), method='highs')

    if result.success:
        optimal_allocation = result.x.reshape(rows, cols)
        return optimal_allocation, result.fun
    else:
        raise ValueError(f"Transportation simplex optimization failed: {result.message}")

# Solving the problem
print("Choose a method to find the initial feasible solution:")
print("1. Northwest Corner Rule")
print("2. Minimum Cost Method")
print("3. Minimum Row Cost Method")
print("4. Vogel's Method")

choice = int(input("Enter your choice (1-4): "))

if choice == 1:
    initial_solution = northwest_corner_rule(supply.copy(), demand.copy())
elif choice == 2:
    initial_solution = minimum_cost_method(cost_matrix, supply, demand)
elif choice == 3:
    initial_solution = minimum_row_cost_method(cost_matrix, supply, demand)
elif choice == 4:
    initial_solution = vogel_method(cost_matrix, supply, demand)
else:
    raise ValueError("Invalid choice")

print("Initial Solution:\n", initial_solution)

optimal_solution, optimal_cost = transportation_simplex(cost_matrix, supply, demand)

print("Optimal Solution:\n", optimal_solution)
print("Optimal Cost:", optimal_cost)